# NLP: INT3406E_41 (2022) - PGS.TS: Nguyen Phuong Thai

---


PROGRAMMING ASSIGNMENT 1: WORD SIMILARITY AND SEMANTIC RELATION CLASSIFICATION
#Vũ Hoàng Long 19020036
#Trần Long Dũng 19021245

In [2]:
!pip install gdown
!git clone https://github.com/NLP-Projects/Word-Similarity.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'Word-Similarity'...
remote: Enumerating objects: 196, done.
remote: Total 196 (delta 0), reused 0 (delta 0), pack-reused 196
Receiving objects: 100% (196/196), 5.81 MiB | 28.74 MiB/s, done.
Resolving deltas: 100% (77/77), done.


# EX1: Cosine similarity:
Given pre-trained embeddings of Vietnamese words, implement a function for calculating cosine similarity between word pairs. Test your program using word pairs in ViSim-400 dataset (in directory Datasets/ViSim-400).

**Dowload pretrained Word2vec Dataset:**
Link download pre-trained word2vec model 
https://drive.google.com/open?id=1z1IDKaZuJXw5g7Yebr1GDq2UfvVR_aGx

In [3]:
!gdown https://drive.google.com/uc?id=1z1IDKaZuJXw5g7Yebr1GDq2UfvVR_aGx

Downloading...
From: https://drive.google.com/uc?id=1z1IDKaZuJXw5g7Yebr1GDq2UfvVR_aGx
To: /content/W2V_150.txt
100% 119M/119M [00:00<00:00, 159MB/s]


**Extract pretrained word2vec data into a Dictionary for searching embedded vectors by key work**

In [4]:
import csv

vec_data_file = '/content/W2V_150.txt' #define path of data file
def word2vec_dictionary(data_file):
  emb_vec_dict = {}
  with open(data_file, 'r') as fd: #read data
      reader = csv.reader(fd)
      for row in reader:
        split_data = row[0].split(' ') #split data by column
        word = split_data[0]
        emb_vec = [] #create new list for containing all the 150 vectors
        for v in split_data[2:-1]: #split 150 column of vectors
          emb_vec.append(float(v)) #parsing the string vector to float number and add to list
        emb_vec_dict[word] = emb_vec #add key word and value of 150 vectors of that word into dictionary
  return emb_vec_dict
word2vec = word2vec_dictionary(vec_data_file) #create new dictionary object

In [5]:
word = 'đẹp_trai'
print(word2vec[word])
print(len(word2vec[word]))

[0.9002099, 0.4414572, 0.3195622, 0.1525354, -0.8680423, -0.9192941, 1.724312, 0.129385, 0.5892586, -0.9284741, -1.090982, 0.3210891, -0.05774014, 0.5338939, -1.180989, 0.5865031, -0.632705, -1.133461, -0.07345625, 0.7923573, -0.1071931, 0.06241727, -0.03073531, 0.2454646, -0.1052362, -3.101819, 1.593211, -1.690924, -1.489635, 1.260306, -0.2791042, -1.138475, -2.131525, -0.6840508, -1.942854, -0.4895685, 1.624937, 0.5780962, 0.02403186, -0.08410149, -0.9141436, -1.183267, -0.3039933, -1.103466, -1.340766, 0.147565, 0.7379212, 0.7285, -1.895763, 1.505164, -1.396337, 0.4565657, -0.3163474, 0.1690939, 3.292289, 0.02980663, -0.3962629, -0.6508451, 0.1695397, 3.389216, -1.18901, 1.98491, 0.5849065, -0.6100891, 0.1037576, 0.6502007, 0.2622414, -0.6728628, 0.5830782, -0.2222768, 0.8332402, -0.8051164, 1.258237, -1.051222, -1.486286, -0.007887463, -0.8877624, 1.526078, -0.6547452, 1.070157, 1.614038, 0.89986, -2.102119, 0.769101, -1.317077, 0.3960995, 0.7461163, 0.3436002, -2.885336, -0.121567

**Define the cosine_similarity function for calculating the cosine similarity between 2 vecs:**

In [6]:
import numpy as np
from scipy import spatial

a = [1,2,3,4]
b = [5,6,7,8]
def cosine_similarity_cal(v1, v2):
  return (2 - spatial.distance.cosine(v1, v2))/2
print(cosine_similarity_cal(a,b))

0.9844319658134832


**Calculate the cosine similarity between 2 word by vec:**

In [7]:
data_file = '/content/Word-Similarity/datasets/ViSim-400/Visim-400.txt'
#data = f = open(data_file, "r")
import csv
i = 0
n_compair = 50
with open(data_file, 'r') as fd:
    reader = csv.reader(fd)
    for row in reader:
      split_data = row[0].split('\t')
      word1 = split_data[0]
      word2 = split_data[1]
      try:
        vec1 = np.array(word2vec[word1])
        vec2 = np.array(word2vec[word2])
        print("Pair-Word: {} - {}, Cosine-similarity: {}".format(word1, word2, cosine_similarity_cal(vec1, vec2)))
      except:
        print('This compair word was not found in dictionary!')
      i+=1
      if i == n_compair:
        break

This compair word was not found in dictionary!
Pair-Word: biến - ngập, Cosine-similarity: 0.497543830265165
Pair-Word: nhà_thi_đấu - nhà, Cosine-similarity: 0.5412615916460588
Pair-Word: động - tĩnh, Cosine-similarity: 0.6385429799341388
Pair-Word: khuyết - ưu, Cosine-similarity: 0.5883993141781336
This compair word was not found in dictionary!
Pair-Word: thủ_pháp - biện_pháp, Cosine-similarity: 0.7011830645971531
Pair-Word: kết_duyên - thành_hôn, Cosine-similarity: 0.7315042010070361
Pair-Word: cấp_tiến - bảo_thủ, Cosine-similarity: 0.6284735044498062
Pair-Word: nước_lớn - nguy_hiểm, Cosine-similarity: 0.5925960112010581
Pair-Word: hoa - nhị, Cosine-similarity: 0.6115348022627427
Pair-Word: bất_lợi - thuận_lợi, Cosine-similarity: 0.7674456514981696
Pair-Word: phân_ly - sum_họp, Cosine-similarity: 0.5388505446519811
This compair word was not found in dictionary!
This compair word was not found in dictionary!
Pair-Word: diễu_hành - tuần_hành, Cosine-similarity: 0.8219417700919267
Pair-W

# EX2: K-nearest words: Given a word w, find k most-similar words of w using the function implemented in question 1.

**Find k nearest text function:**

In [8]:
import pandas as pd
def k_nearest_neighbor(w, v, k = 1):
  similar_vec = [] #create a similar vec list for choosing a nearest vector
  map_v2w = {} #create a dictionary for mapping the vector to word after find the k nearest
  output = {}
  for i in range(len(v)):
    w_vec = word2vec[w] #search the embedding vector of the word
    word_vec = word2vec[v[i]] #search the embedding vector of the word
    csc = cosine_similarity_cal(w_vec, word_vec) #calculate the cosine similarity
    similar_vec.append(csc) #append similar score to list for searching the nearest
    map_v2w[csc] = v[i] #adding the key: vector and value: word to dictionary for quering the word which is nearest
  similar_vec.sort() #sorting by ascending order
  k_nearest_vec = similar_vec[-k:] #quering the k-nearest vector
  for vec in k_nearest_vec:
    word = map_v2w[vec] #mapping the k-nearest vector to word
    output[word] = vec
  return output
#Testing function
k = 10
w = "báo_đốm"
v = ["nước_lớn", "tiền", "nhà_thi_đấu", "kết_duyên", "mèo", "xiếc", "nguy_hiểm", "sóng", 
     "hổ", "sư_tử", "móng_vuốt","bộ_lông", "cá", "ngựa", "nhà_nước"]
print("Input w word: {}".format(w))
df = pd.DataFrame(k_nearest_neighbor(w, v, k), index=[0])
print(df)

Input w word: báo_đốm
   nước_lớn  nguy_hiểm      xiếc      ngựa       cá  móng_vuốt     sư_tử  \
0  0.490419   0.515939  0.589138  0.618687  0.65813   0.667844  0.685771   

         hổ      mèo   bộ_lông  
0  0.715635  0.72033  0.741457  


# EX3: Synonym-antonym classification: Implement a Logistic Regression classifier or a Multi-layer Perceptron classifier for distinguishing synonyms and antonyms.

**Extract 301 Vector: (150 each word) and Sim Score (1 each pair-word) of Synonym and Antonym dataset**

In [9]:
import pandas as pd

ant_file = '/content/Word-Similarity/antonym-synonym set/Antonym_vietnamese.txt'
syn_file = '/content/Word-Similarity/antonym-synonym set/Synonym_vietnamese.txt'

css = []
type_of_word = []
dict_dataset = {}
dict_lists = {}
for i in range(1,301):
  dict_lists['l'+str(i)] = []
with open(ant_file, 'r') as fd:
    for row in fd: 
      split_word = row.split(' ')
      try: #try if dictionary have that word
        word1 = split_word[0].strip()
        word2 = split_word[1].strip()
        vec1 = np.array(word2vec[word1])
        vec2 = np.array(word2vec[word2])
        vec12 = np.concatenate((vec1, vec2))
        css.append(cosine_similarity_cal(vec1, vec2))
        type_of_word.append(0)
        for i in range(300):
            dict_lists['l'+str(i+1)].append(vec12[i])
      except:
        pass
with open(syn_file, 'r') as fd:
    for row in fd:
      split_word = row.split(' ')
      try: #try if dictionary have that word
        word1 = split_word[0].strip()
        word2 = split_word[1].strip()
        vec1 = word2vec[word1]
        vec2 = word2vec[word2]
        vec12 = np.concatenate((vec1, vec2))
        css.append(cosine_similarity_cal(vec1, vec2))
        type_of_word.append(1)
        for i in range(300):
            dict_lists['l'+str(i+1)].append(vec12[i])
      except:
        pass
dict_dataset['CSS'] = css
dict_dataset['Type'] = type_of_word
for i in range(300):
  dict_dataset['vec'+str(i+1)] = dict_lists['l'+str(i+1)]
df = pd.DataFrame(dict_dataset)
df = df.reindex(np.random.permutation(df.index))

df_no_type = df.drop('Type', axis=1)
features = df_no_type.to_numpy()
labels = df['Type'].to_numpy()

In [10]:
print(df)

           CSS  Type      vec1      vec2      vec3      vec4      vec5  \
4450  0.753108     1 -0.374032  4.361281 -0.453997  0.749064  1.222181   
2971  0.559782     1  1.536286  0.698250 -1.203994  1.087432  1.033297   
4165  0.561599     1  0.217570 -1.699579  1.219833 -0.639883  2.039269   
7357  0.616358     1  0.089850 -0.842885 -1.158304 -1.052221 -0.185887   
7661  0.898512     1  0.334460  0.180449  0.065709 -0.751046 -0.822057   
...        ...   ...       ...       ...       ...       ...       ...   
7826  0.836643     1  0.367634 -2.305648 -0.385922 -1.149660  0.859798   
2332  0.680168     1 -0.216281  1.090705 -1.500688  1.100905  0.798344   
1658  0.612497     0  0.161608  0.995400 -0.277558  1.630965  2.345793   
1967  0.562264     1  0.907201  0.761925 -0.986238  1.355984  0.025384   
3900  0.536655     1 -0.725507  0.873804  3.612312  0.289523  0.662060   

          vec6      vec7      vec8  ...    vec291    vec292    vec293  \
4450  1.807540 -0.758798  1.175439  ..

In [11]:
df.to_csv('Word_pair_by_Vec_Data.csv', sep='\t')

**Split data for training and testing**

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(features, labels, test_size=0.30)
print('Training records:', Y_train.size)
print('Val records:', Y_val.size)

Training records: 5598
Val records: 2400


**Testing training with sklearn nn MLP classifier**

In [13]:
# Import MLPClassifer 
from sklearn.neural_network import MLPClassifier

# Create model object
clf = MLPClassifier(hidden_layer_sizes=(64,32),
                    random_state=1,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf.fit(X_train,Y_train)

Iteration 1, loss = 0.40042772
Iteration 2, loss = 0.20636114
Iteration 3, loss = 0.10344609
Iteration 4, loss = 0.04724831
Iteration 5, loss = 0.02496205
Iteration 6, loss = 0.02226124
Iteration 7, loss = 0.02720229
Iteration 8, loss = 0.02761169
Iteration 9, loss = 0.02078475
Iteration 10, loss = 0.01151792
Iteration 11, loss = 0.00948655
Iteration 12, loss = 0.00975757
Iteration 13, loss = 0.00843164
Iteration 14, loss = 0.00741934
Iteration 15, loss = 0.00523630
Iteration 16, loss = 0.00403942
Iteration 17, loss = 0.00144284
Iteration 18, loss = 0.00509128
Iteration 19, loss = 0.00178694
Iteration 20, loss = 0.00370857
Iteration 21, loss = 0.00081636
Iteration 22, loss = 0.00081327
Iteration 23, loss = 0.00070201
Iteration 24, loss = 0.00060100
Iteration 25, loss = 0.00073697
Iteration 26, loss = 0.00063680
Iteration 27, loss = 0.00068360
Iteration 28, loss = 0.00092616
Iteration 29, loss = 0.00137698
Iteration 30, loss = 0.00235290
Iteration 31, loss = 0.00098972
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(64, 32), learning_rate_init=0.01,
              random_state=1, verbose=True)

**TEST THE MODEL WITH VISIM400 and print OUTPUT**

In [21]:
from sklearn import metrics
test_file = '/content/Word-Similarity/datasets/ViCon-400/400_noun_pairs.txt'
#data = f = open(data_file, "r")
import csv
k = 0
syn_ant = {
    "ANT": 0,
    "SYN": 1
}
n_compair = 1
test_data = []
vec = []
css = []
type_of_word = []
dict_testset = {}
dict_lists = {}
p_word1 = []
p_word2 = []
for i in range(1,301):
  dict_lists['l'+str(i)] = []
with open(test_file, 'r') as fd:
      for row in fd:
        if k >= 1:
          split_data = row.split('\t')
          try: #try if dictionary have that word
            word1 = split_data[0].strip()
            word2 = split_data[1].strip()
            types = split_data[2].strip()
            #print(word1, word2, syn_ant[types])
            vec1 = np.array(word2vec[word1])
            vec2 = np.array(word2vec[word2])
            type_of_word.append(syn_ant[types])
            vec12 = np.concatenate((vec1, vec2))
            css.append(cosine_similarity_cal(vec1, vec2))
            p_word1.append(word1)
            p_word2.append(word2)
            for i in range(300):
              dict_lists['l'+str(i+1)].append(vec12[i])
          except:
            pass
        k+=1
dict_testset['Word1'] = p_word1
dict_testset['Word2'] = p_word2
dict_testset['CSS'] = css
dict_testset['Type'] = type_of_word
for i in range(300):
  dict_testset['vec'+str(i+1)] = dict_lists['l'+str(i+1)]
df = pd.DataFrame(dict_testset)
df_only_input = df.drop(['Word1','Word2','Type'], axis=1)
# create test data
test = df_only_input.to_numpy()
test_labels = df['Type'].to_numpy()
# prediction
predicted_y = clf.predict(test[:50])
#print(X_val)
#print(predicted_y)
#Print output
print("Accuracy:")
print("___________________________________________________________________")
print(metrics.classification_report(test_labels[:50], predicted_y))
output_dict = {"Word1" : p_word1[:50],
               "Word2" : p_word2[:50],
               "Cosine Similarity": css[:50],
               "Predicted": predicted_y,
               "True Value": type_of_word[:50]
               }
out_test = pd.DataFrame(output_dict)
print("____________________________________________________________________")
print("1 is Synonym and 0 is Antonym, Output test Visualize:")
print("____________________________________________________________________")
print(out_test)

Accuracy:
___________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      0.97      0.98        32
           1       0.95      1.00      0.97        18

    accuracy                           0.98        50
   macro avg       0.97      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

____________________________________________________________________
1 is Synonym and 0 is Antonym, Output test Visualize:
____________________________________________________________________
           Word1        Word2  Cosine Similarity  Predicted  True Value
0      khoái_lạc      nỗi_đau           0.622883          0           0
1        yếu_kém     sức_mạnh           0.585686          0           0
2     thanh_danh       ô_nhục           0.544493          0           0
3            dây          dọc           0.570831          1           1
4             ba           me           0.4

**Implement the MLP from scratch**

In [15]:
import time
class MultiLayerPerceptron():
  def __init__(self, sizes, epochs=10, l_rate=0.001):
    self.sizes = sizes
    self.epochs = epochs
    self.l_rate = l_rate

    self.params = self.initialization()
  
  def sigmoid(self, x, der=False):
    if der:
      return (np.exp(-x))/((np.exp(-x)+1)**2)
    return 1/(1+np.exp(-x))
  
  def softmax(self, x, der=False):
        exps = np.exp(x - x.max())
        if der:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)

  def ReLu(self, x, der=False):
    if der:
      return np.heaviside(x, 1)
    return np.maximum(x, 0)
    
  def initialization(self):
    input_layer = self.sizes[0]
    hidden_1 = self.sizes[1]
    hidden_2 = self.sizes[2]
    output_layer = self.sizes[3]

    params = {
        'W1':np.random.rand(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
        'W2':np.random.rand(hidden_2, hidden_1) * np.sqrt(1. / hidden_2),
        'W3':np.random.rand(output_layer, hidden_2) * np.sqrt(1. / output_layer)
    }

    return params

  def forward_pass(self, x_train):
    params = self.params

    params['A0'] = x_train

    params['Z1'] = np.dot(params["W1"], params['A0'])
    params['A1'] = self.ReLu(params['Z1'])

    params['Z2'] = np.dot(params["W2"], params['A1'])
    params['A2'] = self.ReLu(params['Z2'])

    params['Z3'] = np.dot(params["W3"], params['A2'])
    params['A3'] = self.sigmoid(params['Z3'])

    return params['A3']

  def backward_pass(self, y_train, output):
    params = self.params
    change_w = {}

    # Calculate W3 update
    error = 2 * (output - y_train) / output.shape[0] * self.softmax(params['Z3'], der=True)
    change_w['W3'] = np.outer(error, params['A2'])

    # Calculate W2 update
    error = np.dot(params['W3'].T, error) * self.ReLu(params['Z2'], der=True)
    change_w['W2'] = np.outer(error, params['A1'])

    # Calculate W1 update
    error = np.dot(params['W2'].T, error) * self.ReLu(params['Z1'], der=True)
    change_w['W1'] = np.outer(error, params['A0'])

    return change_w

  def update_network_parameters(self, changes_to_w):
    for key, value in changes_to_w.items():
            self.params[key] -= self.l_rate * value
  
  def compute_accuracy(self, x_val, y_val):
    predictions = []
    for x, y in zip(x_val, y_val):
      output = self.forward_pass(x)
      pred = np.argmax(output)
      predictions.append(pred == np.argmax(y))
    
    return np.mean(predictions)
  
  def train(self, x_train, y_train, x_val, y_val):
    start_time = time.time()
    for iteration in range(self.epochs):
      for x, y in zip(x_train, y_train):
        output = self.forward_pass(x)
        changes_to_w = self.backward_pass(y, output)
        self.update_network_parameters(changes_to_w)
      
      accuracy = self.compute_accuracy(x_val, y_val)
      print('Epoch: {0}, Time spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
          iteration+1, time.time() - start_time, accuracy * 100
      ))
    
  def predict(self, x_test):
    predictions = []
    for x in (x_test):
      output = self.forward_pass(x)
      predictions.append(np.argmax(output))
    return predictions


**Training . . .**

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

mlp = MultiLayerPerceptron(sizes=[301, 64, 32, 2], epochs = 50, l_rate=0.005)
mlp.train(X_train, Y_train, X_val, Y_val)


Epoch: 1, Time spent: 3.18s, Accuracy: 84.62%
Epoch: 2, Time spent: 4.60s, Accuracy: 82.79%
Epoch: 3, Time spent: 6.02s, Accuracy: 77.33%
Epoch: 4, Time spent: 7.44s, Accuracy: 72.58%
Epoch: 5, Time spent: 8.83s, Accuracy: 70.92%
Epoch: 6, Time spent: 10.22s, Accuracy: 71.62%
Epoch: 7, Time spent: 11.60s, Accuracy: 67.12%
Epoch: 8, Time spent: 13.01s, Accuracy: 69.88%
Epoch: 9, Time spent: 14.48s, Accuracy: 73.96%
Epoch: 10, Time spent: 15.89s, Accuracy: 71.88%
Epoch: 11, Time spent: 17.25s, Accuracy: 71.17%
Epoch: 12, Time spent: 18.67s, Accuracy: 70.12%
Epoch: 13, Time spent: 20.03s, Accuracy: 70.38%
Epoch: 14, Time spent: 21.42s, Accuracy: 70.25%
Epoch: 15, Time spent: 22.79s, Accuracy: 71.17%
Epoch: 16, Time spent: 24.24s, Accuracy: 70.17%
Epoch: 17, Time spent: 25.61s, Accuracy: 70.29%
Epoch: 18, Time spent: 26.98s, Accuracy: 70.29%
Epoch: 19, Time spent: 28.32s, Accuracy: 72.08%
Epoch: 20, Time spent: 29.69s, Accuracy: 72.50%
Epoch: 21, Time spent: 31.06s, Accuracy: 72.71%
Epoch:

**Predict and compute accuracy:**

In [19]:
predict = mlp.predict(test[:10])
output_dict = {"Word1" : p_word1[:10],
               "Word2" : p_word2[:10],
               "Cosine Similarity": css[:10],
               "Predicted": predict,
               "True Value": type_of_word[:10]
               }
out_test = pd.DataFrame(output_dict)
print(metrics.classification_report(out_test['True Value'], predict))
print("____________________________________________________________________")
print("1 is Synonym and 0 is Antonym, Output test Visualize:")
print("____________________________________________________________________")
print(out_test)

              precision    recall  f1-score   support

           0       0.86      0.75      0.80         8
           1       0.33      0.50      0.40         2

    accuracy                           0.70        10
   macro avg       0.60      0.62      0.60        10
weighted avg       0.75      0.70      0.72        10

____________________________________________________________________
1 is Synonym and 0 is Antonym, Output test Visualize:
____________________________________________________________________
          Word1      Word2  Cosine Similarity  Predicted  True Value
0     khoái_lạc    nỗi_đau           0.622883          1           0
1       yếu_kém   sức_mạnh           0.585686          1           0
2    thanh_danh     ô_nhục           0.544493          0           0
3           dây        dọc           0.570831          0           1
4            ba         me           0.475859          0           0
5          ngày    ban_đêm           0.576550          0           

**Result is very bad!!! =>>>>> Underfitting: Model Cannot Fitting**